# Tree search with 0-1 integer programming

This notebook follows from the meeting on 5 Oct, 2021. I think it's a good time to expand the action space and allow random flipping of variables. Before I get make myself crazy again, I want to make sure that it achieves something.



I wish one day they can automatically included these for us.

In [2]:
import numpy as np

import gurobipy as gp 
from gurobipy import GRB

import torch 
import torch.nn as nn
import torch.nn.functional as F

## Gurobi to actually solve the 0-1 integer programs

In [3]:
def gurobi_optimal(A, b, c, *, num_solution_limit = 32):
    m, n = A.shape
    assert b.shape == (m, ) and c.shape == (n, )

    ip = gp.Model()
    x_var = ip.addMVar(n, lb = 0.0, ub = float('inf'), vtype = GRB.BINARY, name = 'x')
    ip.addConstr(A @ x_var <= b)
    ip.setObjective(c @ x_var, GRB.MAXIMIZE)
    ip.setParam('OutputFlag', 0)
    ip.setParam('PoolSearchMode', 2)
    ip.setParam('PoolSolutions', num_solution_limit)
    ip.optimize()

    solutions = []
    obj = c @ np.round(x_var.x)
    for i in range(ip.SolCount):
        ip.setParam('SolutionNumber', i)
        sol = np.array(ip.xn).round().astype(bool)
        if c @ sol < obj:
            break
        else:
            solutions.append(sol)
    return obj, np.stack(solutions)

## Data and dataset
### Problems to be explored
Please refer to the Dataset part for details.

#### Set Cover Problems

In [3]:
from problems.setcover import *
problem_func = generate_setcover
encoding_func = setcover_encoding
def size_func():
    return {'nrows': 100, 'ncols': 200, 'density': np.random.uniform(0.1, 0.25)}


#### Verify the problem size
- $A\in \mathbb{R}^{m\times n}$
- $b\in \mathbb{R}^m$
- $c\in \mathbb{R}^n$

In [5]:
def valid_dim(A, b, c, x = None):
    m, n = A.shape 
    return b.shape == (m, ) and c.shape == (n,) and (x is None or x.shape == (n, ))

### Encoding of the problems

### Dataset


In [6]:
class BinaryIPDataset(torch.utils.data.Dataset):
    '''
    Generate a problem instance with a random, non-terminal assignment
    '''
    def __init__(self, *, problem_func, size_func, assignment_func, encoding_func):
        '''
        problem_func: problem to be explored
        size_func: size as a input parameter to the problem_func
        problem_func(**size_func()) should work
        
        assignment_func: used to generate an assignemnt for the given problem
        encoding_func: 
        '''
        super().__init__()
        self.problem_func = problem_func
        self.size_func = size_func
        self.assignment_func = assignment_func
        self.encoding_func = encoding_func

    def __len__(self):
        raise NotImplementedError # not needed

    def __getitem__(self, index:int):
        # get a random instance in the class of problem with a random size
        A, b, c = self.problem_func(**self.size_func())

        # get a meaningful but random variable assignemnt
        x_assignment = self.assignment_func(A, b, c)
        
        # get a random intermediate solution
        _, solutions = gurobi_optimal(A, b, c)

        difference = np.logical_xor(x_assignment, solutions)
        dist = difference.sum(axis = 1)
        min_dist_difference = torch.tensor(difference[dist == dist.min()])
        
        u, v, e = self.encoding_func(A, b, c, x_assignment)

        return u, v, e, min_dist_difference

#### Totally random assignment function

But first draw uniformly the number of 1-valued variables, then the number of variables.



In [7]:
def random_assignment(A, b, c):
    m, n = A.shape
    num_ones = np.random.randint(n + 1)
    assignment = np.zeros_like(c, dtype = bool)
    assignment[np.random.choice(n, size = num_ones, replace = False)] = True
    return assignment

## Model

In [8]:
def dense_stack(*args, output_relu = True):
    seq = nn.Sequential()
    for i in range(1, len(args)):
        seq.add_module(f'dense {i-1}', nn.Linear(args[i-1], args[i]))
        if i < len(args) - 1 or output_relu:
            seq.add_module(f'relu {i-1}', nn.ReLU())
    return seq

class HalfConvolution(nn.Module):
    '''
    input: a bipartite graph
    u: features of the nodes on the same side (U, F) 
    v: features of the nodes on the opposite side (V, G)
    e: featuers of of the edges (V, U, H)

    g_args (F+G+H, ..., D)
    f_args (F+D, ...)
    '''
    def __init__(self, *, f_args, g_args):
        super().__init__()
        self.g = dense_stack(*g_args)
        self.f = dense_stack(*f_args)
    def forward(self, u, v, e):
        U, F = u.shape
        V, G = v.shape
        _, _, H = e.shape
        assert e.shape == (V, U, H)
        
        g_out, _ = self.g(torch.cat([u.unsqueeze(-3).expand(V, U, F), v.unsqueeze(-2).expand(V, U, G), e], axis = -1)).max(axis = -3) # (V, U, D) to (U, D)
        out = self.f(torch.cat([u, g_out], axis = -1))
        return out

class HalfConvolutionModel(nn.Module):
    '''
    input: same as HalfConvolution

    variable_args 
    constraits_args
    final_args
    '''
    def __init__(self, *, v_feats: int, c_feats: int, e_feats: list, g_hidden_neurons: list, f_hidden_neurons: list, out_neurons: list):
        super().__init__()
        self.half_conv = HalfConvolution(
            g_args = [v_feats+c_feats+e_feats] + g_hidden_neurons,
            f_args = [v_feats + g_hidden_neurons[-1]] + f_hidden_neurons
        )
        self.out = dense_stack(f_hidden_neurons[-1], *out_neurons, output_relu = False)
    
    def forward(self, u, v, e):
        out = self.half_conv(u, v, e)  
        return self.out(out)
    
    def predict(self, A, b, c, x):
        raise NotImplementedError

## Training
### Set up the dataset and the model

In [8]:
model = HalfConvolutionModel(v_feats= 5, c_feats= 5, e_feats=1, g_hidden_neurons=[64, 64], f_hidden_neurons=[64, 64], out_neurons=[64, 1])

In [9]:
ds = BinaryIPDataset(problem_func = problem_func, size_func = size_func, assignment_func = random_assignment, encoding_func= encoding_func)

### Training configurations

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = np.sqrt(0.1))
Loss = torch.nn.BCEWithLogitsLoss(reduction = 'none')

num_batch, batch_size = 100, 16

### Training loop

In [11]:
for i in range(num_batch):
    optimizer.zero_grad()
    loss = torch.tensor(0.0)
    for j in range(batch_size):
        u, v, e, min_dist_diff = ds[j]
        N_sols, _ = min_dist_diff.shape

        scores = model(u, v, e).t().expand(N_sols, -1) # (n, 1) -> (1, n) -> (N_sols, n)

        _loss = Loss(input = scores, target = min_dist_diff.float()).mean(axis = 1)
        loss = loss + _loss.min() / batch_size 
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    print(f'Batch {i}, loss: {loss.item():.3f}')

In [13]:
torch.save(model, './model.pt')

### Prediction

In [14]:
model = torch.load('./model.pt')

In [15]:
# Test set generating
def generate_test(len_test=1026, problem_func=problem_func):
    problems, objectives = [], []
    for i in range(len_test):
        A, b, c = problem_func(**size_func())
        problems.append((A, b, c))
        obj, _ = gurobi_optimal(A, b, c)
        objectives.append(obj)
    
    return problems, objectives

In [41]:
p, o = generate_test(100)

In [62]:
def estimator(A, b, c, x, encoding_func=encoding_func):
    if model.training:
        model.eval()
    with torch.no_grad():
        u, v, e = encoding_func(A, b, c, x)
        scores = model(u, v, e).detach().numpy().flatten()
    return scores

In [73]:
import collections
def tree_search(A, b, c, objective, estimator, threshold=0.5, max_children=2, num_step_limit = np.inf):
    """
    ## Parameter

    `A, b, c, objective`: problem

    `estimator`: score function
    
    `threshold`: which score to choose
    
    `max_children`: children of each node

    `num_step_limit`: max times to run
    """
    m, n = A.shape
    assert b.shape == (m, ) and c.shape == (n, )
    tree = collections.deque()

    step_count = 0
    run = True
    best_x = np.zeros_like(c, dtype = bool)
    current_obj = 0
    while step_count < num_step_limit and run:  
        if tree:
            x = tree.popleft()
        else:
            x = np.zeros_like(c, dtype = bool)

        score = estimator(A, b, c, x)
        score[score < threshold] = 0
        indices = np.argsort(score, kind='quicksort')[-1:-(max_children+1):-1]
        for index in indices:
            if score[index] == 0:
                break
            else: 
                x[index] = ~x[index]
                valid = (A @ x <= b).all()
                if valid:
                    tree.append(x)
                    # When to stop?
                    if c @ x >= current_obj:
                        best_x = x.copy()
                        current_obj = c @ x
                    if c @ x == objective:
                        run = False
                        best_x = x.copy()
        
        if not tree and step_count != 0:
            run = False

        step_count += 1
        # print("Current tree size: {}".format(len(tree)))

    return best_x

In [75]:
for problem, objective in zip(p, o):
    A, b, c = problem
    best_x = tree_search(A, b, c, objective, estimator=estimator, threshold=0.0, num_step_limit=4096)
    print("Objective %d, Result %d" % (objective, c @ best_x))
    print(b - A @ best_x)

Objective 1723, Result 1462
[2. 0. 1. 3. 0. 3. 2. 1. 2. 1. 2. 3. 5. 2. 0. 1. 3. 2. 0. 0.]
Objective 2001, Result 1966
[0. 1. 2. 2. 0. 0. 0. 0. 1. 2. 1. 1. 1. 0. 0. 1. 1. 0. 3. 0.]
Objective 1791, Result 1375
[1. 6. 0. 1. 5. 2. 2. 1. 1. 1. 7. 3. 1. 2. 1. 1. 2. 3. 2. 2.]
Objective 1791, Result 1781
[1. 0. 1. 2. 0. 0. 1. 0. 0. 0. 0. 2. 0. 2. 1. 1. 0. 0. 0. 0.]
Objective 1651, Result 1612
[0. 2. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 2. 2. 1. 1. 2. 0. 1. 1.]
Objective 1873, Result 1709
[0. 1. 1. 2. 0. 0. 2. 1. 2. 3. 1. 2. 0. 1. 1. 1. 2. 2. 1. 2.]
Objective 1682, Result 1644
[0. 0. 1. 0. 1. 2. 2. 0. 0. 0. 0. 2. 1. 0. 0. 0. 0. 1. 2. 1.]
Objective 1651, Result 1551
[1. 2. 0. 1. 0. 0. 0. 0. 3. 0. 2. 1. 0. 0. 0. 2. 1. 0. 0. 2.]
Objective 2095, Result 1971
[1. 1. 1. 0. 2. 1. 3. 0. 0. 0. 1. 1. 3. 3. 0. 0. 0. 2. 1. 2.]
Objective 1948, Result 1880
[1. 0. 0. 1. 2. 2. 2. 3. 3. 1. 1. 0. 4. 0. 2. 3. 0. 1. 0. 2.]
Objective 1745, Result 1729
[1. 0. 1. 0. 1. 1. 2. 2. 1. 0. 0. 1. 0. 0. 1. 0. 3. 0. 2. 0.]
Objective 